### Building polymers with the supramolecular tooklit (stk)

In [1]:
import stk
import rdkit, rdkit.Chem as rdkit

## 1. Building a co-polymer

In [2]:
'''
stk is a supramolecular buildier which takes building blocks and can join them 
together to form supramolecular structures with different topologies.
In our case, we are going to build polymers (i.e. a supramolecules with a 'linear' topology).
To do so, we will need to provide information about our building blocs (a, b) and the polymer
characteristics (length, sequence etc.)
'''

# specifying the building blocks a and b using rdkit directly from smiles strings
a = rdkit.MolFromSmiles('Brc1ccc(Br)cc1')
rdkit.AddHs(a)
rdkit.AllChem.EmbedMolecule(a, rdkit.AllChem.ETKDG())

b = rdkit.MolFromSmiles('Brc1ccc(Br)nc1')
rdkit.AddHs(b)
rdkit.AllChem.EmbedMolecule(b, rdkit.AllChem.ETKDG())

# convert our rdkit 'mol' objects to stk 'StructUnit2' objects (stk can only regognise these when building)
A = stk.StructUnit2.rdkit_init(a, 'bromine')
B = stk.StructUnit2.rdkit_init(b, 'bromine')

# with our StructUnit2s in hand, we can now build the polymer.
# Here, we habe 2 building blocks (A, B), a repeat unit sequence "AB" and a length of 8 repeat units.
polymer = stk.Polymer([A, B], stk.Linear("AB", [0, 0], n=8))
# the list [0, 0] indicates the orientation of each building block (important if one is asymmetric)

# the 'polymer' we created above now only has a rough gemoetry, we can clean that up using the
# ETKDG method for structural optimisation/embedding within rdkit
stk.rdkit_ETKDG(polymer)

# lastly we write our new polymer structure to a '*.mol' file (you can use avogadro to look at this)
polymer.write('isabelle-test.mol')

## 2. Building a homopolymer

In [3]:
# same sort of thing again only now we will consider the simpler case of a 
# homopolymer (only one type of building block)

a = rdkit.MolFromSmiles('Brc1cnc(Br)cn1')
rdkit.AddHs(a)
rdkit.AllChem.EmbedMolecule(a, rdkit.AllChem.ETKDG())

A = stk.StructUnit2.rdkit_init(a, 'bromine')

polymer = stk.Polymer([A], stk.Linear("A", [0], n=8))

stk.rdkit_ETKDG(polymer)

polymer.write('isabelle-test.mol')

## 3. Performing a conformer search

In [7]:
'''
Thanks to their many rotatable bonds, polymers usually have many conformers (different orientations
of the roatable bonds). We may then wish to onlu consider the lowest energy among these. To obtain this, 
we must perform a conformer search. The function below does exactly this, taking advantage of inbuilt
functions of rdkit.
'''

def conformer_search(mol, nconfs):
    
    confs = rdkit.AllChem.EmbedMultipleConfs(mol, nconfs, rdkit.AllChem.ETKDG()) 
    rdkit.SanitizeMol(mol)    
    
    lowest_energy = 1000000000
    for conf in confs:
        ff = rdkit.AllChem.MMFFGetMoleculeForceField(mol, rdkit.AllChem.MMFFGetMoleculeProperties(mol), confId=conf)
        ff.Initialize()
        energy = ff.CalcEnergy()
    
        if energy < lowest_energy:
            lowest_energy = energy
            lowest_conf = conf
            print('conformer:', conf, 'energy:', energy)
    
    rdkit.MolToMolFile(mol, 'conformer-search-output.mol', confId=lowest_conf)
    
conformer_search(polymer.mol, 300) # when we call the function, we see the lowest conformer as it is found

conformer: 0 energy: 708.8462641397347
conformer: 1 energy: 693.3771722394251
conformer: 2 energy: 655.6824426119254
conformer: 37 energy: 652.5106814552204
conformer: 84 energy: 628.4988511988192


## 4. Wrapping polymer building into a function#

In [5]:
''' 
Normally, we don't want a lot of straggling code. Below is an example of the polymer building process in 
part 1 wrapped into a function, which takes 2 arguments: smiles1 and smiles2. These represent the 
monomer building blocks that will be used to make the polymer.
'''

def build_polymer(smiles1, smiles2):
    
    a = rdkit.MolFromSmiles(smiles1)
    rdkit.AddHs(a)
    rdkit.AllChem.EmbedMolecule(a, rdkit.AllChem.ETKDG())

    b = rdkit.MolFromSmiles(smiles2)
    rdkit.AddHs(b)
    rdkit.AllChem.EmbedMolecule(b, rdkit.AllChem.ETKDG())

    A = stk.StructUnit2.rdkit_init(a, 'bromine')
    B = stk.StructUnit2.rdkit_init(b, 'bromine')

    polymer = stk.Polymer([A, B], stk.Linear("AB", [0, 0], n=4))

    stk.rdkit_ETKDG(polymer)

    polymer.write('isabelle-test.mol')
    
    return polymer

In [6]:
 build_polymer('Brc1ccc(Br)cc1', 'Brc1ccc(Br)cc1') # here we call the above function which builds a polymer

Polymer(building_blocks=["StructUnit2 ['bromine', 'InChI=1S/C6H4Br2/c7-5-1-2-6(8)4-3-5/h1-4H']", "StructUnit2 ['bromine', 'InChI=1S/C6H4Br2/c7-5-1-2-6(8)4-3-5/h1-4H']"], topology=Linear(ends='h', n=4, orientation=[0, 0], repeating_unit='AB'))